In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
train = pd.read_csv("train.csv")
validation = pd.read_csv("validation.csv")
test = pd.read_csv("test.csv")

In [3]:
#Eliminamos la primera columna, generada al pasar el conjunto de datos a formato csv.
train=train.drop(columns='Unnamed: 0')
validation=validation.drop(columns = 'Unnamed: 0')
test=test.drop(columns='Unnamed: 0')

In [4]:
porcentaje_mantener = 0.0001


train_reducido_udp = train[train['proto'] == 'UDP'].groupby('category').sample(frac=porcentaje_mantener)
train_reducido_tcp = train[train['proto'] == 'TCP'].groupby('category').sample(frac=porcentaje_mantener)

train_reducido = pd.concat([train_reducido_udp, train_reducido_tcp])

validation_reducido_udp = validation[validation['proto'] == 'UDP'].groupby('category').sample(frac=porcentaje_mantener)
validation_reducido_tcp = validation[validation['proto'] == 'TCP'].groupby('category').sample(frac=porcentaje_mantener)

validation_reducido = pd.concat([validation_reducido_udp, validation_reducido_tcp])

test_reducido_udp = test[test['proto'] == 'UDP'].groupby('category').sample(frac=porcentaje_mantener)
test_reducido_tcp = test[test['proto'] == 'TCP'].groupby('category').sample(frac=porcentaje_mantener)

test_reducido = pd.concat([test_reducido_udp, test_reducido_tcp])

In [5]:
train_reducido['proto'].replace({'TCP': 0, 'UDP': 1}, inplace=True)
validation_reducido['proto'].replace({'TCP': 0, 'UDP': 1}, inplace=True)
test_reducido['proto'].replace({'TCP': 0, 'UDP': 1}, inplace=True)

In [6]:
# Separar características y etiquetas para conjunto de entrenamiento
X_train_reducido = train_reducido.drop(columns=['category'])
y_train_reducido = train_reducido['category']

# Separar características y etiquetas para conjunto de validación
X_val_reducido = validation_reducido.drop(columns=['category'])
y_val_reducido = validation_reducido['category']

# Separar características y etiquetas para conjunto de prueba
X_test_reducido = test_reducido.drop(columns=['category'])
y_test_reducido = test_reducido['category']

In [7]:
#Normalización del conjunto de datos reducido
norm = StandardScaler()

X_train_reducido = norm.fit_transform(X_train_reducido)
X_val_reducido = norm.fit_transform(X_val_reducido)
X_test_reducido = norm.fit_transform(X_test_reducido)

In [10]:
from qiskit_ibm_runtime import Session, SamplerV2, EstimatorV2
from qiskit.providers.fake_provider import Fake127QPulseV1
from qiskit_ibm_runtime.fake_provider import FakeBrisbane 
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, QasmSimulator
#from qiskit_aer.primitives import SamplerV2

from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap

num_qubits = X_train_reducido.shape[1]

backend = AerSimulator(method='matrix_product_state', device='CPU')
session = Session(backend=backend)
pass_manager = generate_preset_pass_manager(optimization_level=1, target=backend.target)

sampler = SamplerV2(mode=session)
fidelity = ComputeUncompute(sampler=sampler, pass_manager=pass_manager)

feature_map = ZZFeatureMap(feature_dimension=num_qubits)
qk = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

In [11]:
from qiskit_machine_learning.algorithms import QSVC
import time

qsvc = QSVC(quantum_kernel=qk)

start = time.time()
qsvc.fit(X_train_reducido, y_train_reducido)
stop = time.time()

train_time = stop - start

start = time.time()
qsvc_score = qsvc.score(X_test_reducido, y_test_reducido)
stop = time.time()

test_time = stop - start

print(f"Training Time: {train_time}")
print(f"Testing Time: {test_time}")
print(f"Total Time: {train_time + test_time}")
print(f"QSVC classification test score: {qsvc_score}")

AlgorithmError: 'Sampler job failed!'